# 基础思路（baseline）

## 步骤1：更新或安装所需环境

In [1]:
!pip install --upgrade modelscope requests urllib3 tqdm pandas
!apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip




debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 5.)
debconf: falling back to frontend: Readline


## 步骤2：下载数据集

In [2]:
!axel -n 12 -a https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv

Initializing download: https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv
ERROR 403: Forbidden.


## 步骤3：构建模型（使用Qwen2-7B-Instruct）

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen2-7B-Instruct")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-12 18:33:46,203 - modelscope - INFO - Got 14 files, start to download ...
Processing 14 items:   0%|          | 0/14 [00:00<?, ?it/s]




























Processing 14 items:   7%|▋         | 1/14 [00:00<00:05,  2.37it/s]


Processing 14 items:  14%|█▍        | 2/14 [00:00<00:03,  3.78it/s]








Processing 14 items:  29%|██▊       | 4/14 [00:00<00:01,  7.07it/s]
























Processing 14 items:  36%|███▌      | 5/14 [00:00<00:01,  5.98it/s]





































































Processing 14 items:  50%|█████     | 7/14 [00:01<00:01,  5.91it/s]


































Processing 14 items:  57%|█████▋    | 8/14 [00:01<00:01,  5.79it/s]


























































Processing 14 items:  64%|██████▍   | 9/14 [00:01<00:01,  4.77it/s]


























Processing 14 items:  71%|███████▏  | 10/14 [00:01<00:00,  4.94it/s]


































































## 步骤4：读取数据集

In [ ]:
import pandas as pd
test = pd.read_csv('./test_input.csv', header=None)

In [ ]:
# 查看数据集大小
print(f"数据集的大小为: {test.shape[0]}\n前50条数据如下：\n")

# 查看前50条赛事数据集（赛题要求根据每行句子，给出5个可能匹配的成语）
for test_prompt in test[0].values[:50]:
    print(test_prompt)

## 步骤5：输出成语

In [ ]:
from tqdm import tqdm
import os

i = 1
# 假设 test 是一个 DataFrame
# 遍历测试数据集的第一项的值，目的是生成与给定句子最相关的五个成语
for test_prompt in tqdm(test[0].values, total=len(test[0].values), desc="处理进度"):
    i = i + 1
    # 构造提示信息，要求模型输出与句子最相关的五个成语
    prompt = f"把下面句子总结为一个成语：{test_prompt}"

    # 初始化一个长度为5的列表，填充默认成语“同舟共济”
    words = ['同舟共济'] * 5

    # 构建聊天消息格式，用于提示模型进行生成
    messages = [
        {"role": "user", "content": prompt},
    ]
    # 应用聊天模板对消息进行处理，准备模型输入
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # 对输入文本进行编码，准备模型输入数据
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    # 生成回答，限制最大生成长度
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=50,
        num_beams=1,
        do_sample=False
    
        
    )
    # 提取模型输出，去除输入部分
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    # 解码模型输出，去除特殊标记
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 清理回答文本，确保格式统一
    response = response.replace('\n', ' ').replace('、', ' ')
    # 提取回答中的成语，确保每个成语长度为4且非空
    words = [x for x in response.split() if len(x) == 4 and x.strip() != '']
    
    

    # 如果生成的成语列表长度不满足要求（即20个字符），则使用默认成语列表
    if len(' '.join(words).strip()) != 24:
        words = ['同舟共济'] * 5

    # 将最终的成语列表写入提交文件
    with open('submit.csv', 'a+', encoding='utf-8') as up:
        up.write(' '.join(words) + '\n')

    
    # 查看阶段性结果
    if i % 50 == 0:
        tqdm.write(f"大模型第{i}次返回的结果是：\n   {response}\n")
        tqdm.write(f"submit.cvs第{i}行输出结果：\n   {words}\n")
    
    # 为了尽快拿到结果，我们暂时仅获得500个结果（如果有时间的话，可以删除这两行）
    if i == 10:
        break

print('submit.csv 已生成')